Kurulum hücresi

In [7]:
%%capture
!pip install --no-deps fast-plaid fastkmeans pdf2image colpali-engine --quiet
!apt-get install -y poppler-utils
!pip install pdf2image
!pip install pytesseract
!pip install gradio==5.37.0
!pip install transformers accelerate

Model ve Yardımcı Kütüphane Tanımı

In [2]:
import torch
import pytesseract
import gradio as gr
import time
import hashlib
import pickle
import os
from PIL import Image
from pdf2image import convert_from_path
from transformers.utils.import_utils import is_flash_attn_2_available
from colpali_engine.models import ColQwen2, ColQwen2Processor
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM

Model ve Processor Yükleme

In [3]:
model_name = "vidore/colqwen2-v1.0"

model = ColQwen2.from_pretrained(
    model_name,
    torch_dtype=torch.float32,
    device_map="cpu",
    attn_implementation=None
).eval()

processor = ColQwen2Processor.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/392 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

video_preprocessor_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

chat_template.json: 0.00B [00:00, ?B/s]

Model ve tokenizer'ı yükle

In [4]:
llm_model_id = "microsoft/Phi-3-mini-4k-instruct"
tokenizer = AutoTokenizer.from_pretrained(llm_model_id)
llm_model = AutoModelForCausalLM.from_pretrained(
    llm_model_id,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Cevap Fonksiyonu

In [5]:
def answer_with_llm(query, context_text, max_tokens=300):
    prompt = f"""
You are a helpful assistant. Based on the following document extract, answer the user's question.
Respond in the same language as the question. If the question is in Turkish, answer in Turkish. If it's in English, answer in English.
Give a concise and well-structured answer and always finish the sentences.
Only answer the given question. Do not generate or answer follow-up questions.

Context:
{context_text}

Question: {query}
Answer:"""

    inputs = tokenizer(prompt, return_tensors="pt").to(llm_model.device)

    with torch.no_grad():
        output = llm_model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            temperature=0.5,
            top_p=0.5,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    full_output = tokenizer.decode(output[0], skip_special_tokens=True)
    answer = full_output[len(prompt):].strip()

    if not answer.endswith((".", "!", "?")):
        answer += " ..."

    return answer

Gradio


In [ ]:
CACHE_PATH = "embedding_cache.pkl"

if os.path.exists(CACHE_PATH):
    with open(CACHE_PATH, "rb") as f:
        embedding_cache = pickle.load(f)
else:
    embedding_cache = {}

def get_pdf_hash(file_bytes):
    return hashlib.sha256(file_bytes).hexdigest()

def run_colpali_ocr(file, query):
    global model, processor

    start_total = time.time()

    # PDF hash
    with open(file.name, "rb") as f:
        file_bytes = f.read()
    file_hash = get_pdf_hash(file_bytes)

    start_pdf = time.time()
    if file.name.endswith(".pdf"):
        images = convert_from_path(file.name, dpi=100)
    else:
        images = [Image.open(file)]
    end_pdf = time.time()

    # Embedding
    start_embed = time.time()
    if file_hash in embedding_cache:
        doc_embeddings = embedding_cache[file_hash]
        used_cache = True
    else:
        dataloader = DataLoader(
            dataset=images,
            batch_size=1,
            shuffle=False,
            collate_fn=lambda x: processor.process_images(x),
        )

        doc_embeddings = []
        for batch in dataloader:
            with torch.no_grad():
                batch = {k: v.to(model.device) for k, v in batch.items()}
                embeds = model(**batch)
            doc_embeddings.extend(list(torch.unbind(embeds.to("cpu"))))

        embedding_cache[file_hash] = doc_embeddings
        with open(CACHE_PATH, "wb") as f:
            pickle.dump(embedding_cache, f)
        used_cache = False
    end_embed = time.time()

    # Sorgu Embed
    start_query = time.time()
    batch_queries = processor.process_queries([query]).to(model.device)
    with torch.no_grad():
        query_embeddings = model(**batch_queries)
    end_query = time.time()

    # Skorlar
    start_score = time.time()
    num_pages = len(doc_embeddings)
    plaid_index = processor.create_plaid_index(doc_embeddings)
    scores = processor.get_topk_plaid(query_embeddings, plaid_index, k=num_pages)
    end_score = time.time()

    # Skorları sırala
    page_scores_with_page_num = [(score, doc_idx + 1) for doc_idx, score in scores[0][0]]
    sorted_page_scores = sorted(page_scores_with_page_num, key=lambda x: -x[0])
    top3 = [(page_num - 1, score) for score, page_num in sorted_page_scores[:3]]

    # En iyi 3 sayfanın görselleri ve skor çıktısı
    results = ""
    best_images = []
    for rank, (page_idx, score) in enumerate(top3):
        results += f"{rank+1}. Sayfa: {page_idx+1} - Skor: {score:.4f}\n"
        best_images.append(images[page_idx])

    # Tüm sayfa skorları yüksekten düşüğe
    all_scores = "--- Tüm Sayfa Skorları (Yüksekten Düşüğe) ---\n"
    for score, page_num in sorted_page_scores:
        all_scores += f"Sayfa {page_num}: Skor {score:.4f}\n"

    # Zamanlar
    timing_info = "\n--- İşlem Süreleri (saniye) ---\n"
    timing_info += f"PDF/Görsel Dönüştürme: {end_pdf - start_pdf:.2f}s\n"
    timing_info += f"Embedding: {end_embed - start_embed:.2f}s {'(cache kullanıldı)' if used_cache else ''}\n"
    timing_info += f"Sorgu Embed: {end_query - start_query:.2f}s\n"
    timing_info += f"Skor Hesaplama: {end_score - start_score:.2f}s\n"
    timing_info += f"Toplam Süre: {time.time() - start_total:.2f}s\n"

    # LLM ile yanıt üret (1. en iyi eşleşen sayfa)
    llm_start = time.time()
    first_page_idx = top3[0][0]
    ocr_text = pytesseract.image_to_string(images[first_page_idx])
    llm_answer = answer_with_llm(query, ocr_text)
    llm_end = time.time()

    return results + timing_info, best_images, all_scores, f"{llm_answer}\n\n⏱️ LLM Süresi: {llm_end - llm_start:.2f}s"


demo = gr.Interface(
    fn=run_colpali_ocr,
    inputs=[
        gr.File(label="PDF veya Görsel dosyası", file_types=[".pdf", ".png", ".jpg", ".jpeg"]),
        gr.Textbox(label="Sorgu metni")
    ],
    outputs=[
        gr.Textbox(label="Skor Sonuçları ve Süreler"),
        gr.Gallery(label="En iyi Eşleşen Sayfalar"),
        gr.Textbox(label="Tüm Sayfa Skorları"),
        gr.Textbox(label="LLM Cevabı")
    ],
    title="ColPali + Phi-3-mini",
    description="ColPali ile en iyi eşleşen sayfaları bul, LLM ile doğal dilde cevabını al."
)
demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://6a0680d5558bce784a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
